# Kappa score calculation

In [2]:
# Project imports
import pandas as pd
import numpy as np
from sklearn.metrics import cohen_kappa_score

# Project variables
inputFile = "input.csv"

##### Import CSV
Import the CSV and drop collums that are not relevant for this script.

In [73]:
data = pd.read_csv(inputFile)

# Drop the index, thread and email ID
data.drop("#", inplace=True, axis=1)
data.drop("Tread Id", inplace=True, axis=1)
data.drop("Email ID", inplace=True, axis=1)
data.drop("Agreement?", inplace=True, axis=1)
data.drop("Decided", inplace=True, axis=1)

# Drop NaN value rows
data.dropna(inplace=True)

# Display
data.head(5)

,Marten,Rick,Krishan
0,not-ak,not-ak,not-ak
1,technology,not-ak,"technology,existence,existence-behavioral"
2,technology,not-ak,"technology,existence,existence-behavioral"
3,not-ak,not-ak,not-ak
4,process,process,process


In [4]:
# Get unique labels used in the input set.
# Used to verify validate validity of the data.
uniques = pd.unique(data.values.ravel("K"))
uniques = pd.Series(uniques).map(lambda x: x.split(',')).explode().unique()

# Display unqiues
for label in uniques:
    print(label)

not-ak
technology
process
existence
existence-structural
existence-behavioral
property
existence-behaviour


##### Convert to AK and Not-AK
Used for comparisons of Kappa score for checking if the group is on the same line for this property.

In [83]:
# Hierachy for most important to least important labels
labelHierachy = ["technology", "process", "property", "existence", "not-ak"]

def f(row):
    # This can be nicer in one loop probably (or with a 3 one-liners) but I dont really care tbh.
    for label in labelHierachy:
        if label in row[0]:
            row[0] = label;
            break;
    for label in labelHierachy:
        if label in row[1]:
            row[1] = label;
            break;
    for label in labelHierachy:
        if label in row[2]:
            row[2] = label;
            break;
    return row

copy = data.copy()

copy.apply(f, axis=1)
print(copy)

         Marten        Rick     Krishan
0        not-ak      not-ak      not-ak
1    technology      not-ak  technology
2    technology      not-ak  technology
3        not-ak      not-ak      not-ak
4       process     process     process
5       process     process     process
6        not-ak      not-ak      not-ak
7        not-ak      not-ak      not-ak
8        not-ak      not-ak      not-ak
9        not-ak      not-ak      not-ak
10       not-ak      not-ak      not-ak
11       not-ak      not-ak      not-ak
12       not-ak      not-ak      not-ak
13       not-ak      not-ak      not-ak
14       not-ak      not-ak      not-ak
15       not-ak      not-ak      not-ak
16       not-ak      not-ak      not-ak
17       not-ak      not-ak      not-ak
18       not-ak      not-ak      not-ak
19      process      not-ak      not-ak
20       not-ak      not-ak      not-ak
21       not-ak      not-ak      not-ak
22       not-ak      not-ak      not-ak
23       not-ak      not-ak     process


In [86]:
def calculateKappa(df, start=0, end=None):
    if end == None:
        end = df.shape[0]

    colLength = df.shape[1]
    output = pd.DataFrame(columns=df.columns, index=df.columns)

    output

    for rIdx in range(0, colLength):
        part1 = df.iloc[start:end, rIdx]
        for cIdx in range(0, colLength):
            part2 = df.iloc[start:end, cIdx]
            output.iloc[cIdx, rIdx] = cohen_kappa_score(part1, part2)

    # Print the output
    print("--------------------------------------------")
    print(f"Created from rows {start} till {end}")
    print(output.to_latex())

calculateKappa(copy, end=50)
calculateKappa(copy, start=51, end = 103)

0         not-ak
1     technology
2     technology
3         not-ak
4        process
5        process
6         not-ak
7         not-ak
8         not-ak
9         not-ak
10        not-ak
11        not-ak
12        not-ak
13        not-ak
14        not-ak
15        not-ak
16        not-ak
17        not-ak
18        not-ak
19       process
20        not-ak
21        not-ak
22        not-ak
23        not-ak
24       process
25        not-ak
26        not-ak
27       process
28       process
29       process
30        not-ak
31        not-ak
32        not-ak
33        not-ak
34        not-ak
35        not-ak
36        not-ak
37        not-ak
38        not-ak
39     existence
40       process
41        not-ak
42     existence
43     existence
44     existence
45     existence
46        not-ak
47       process
48        not-ak
49        not-ak
Name: Marten, dtype: object 0         not-ak
1     technology
2     technology
3         not-ak
4        process
5        process
6         not-ak
7  

C:\Users\rickt\AppData\Local\Temp\ipykernel_10508\4178943380.py:20: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(output.to_latex())
C:\Users\rickt\AppData\Local\Temp\ipykernel_10508\4178943380.py:20: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(output.to_latex())
